#### API定义：

应用程序接口（英语：Application Programming Interface，简称：API），又称为应用编程接口，就是软件系统不同组成部分衔接的约定。由于近年来软件的规模日益庞大，常常需要把复杂的系统划分成小的组成部分，编程接口的设计十分重要。程序设计的实践中，编程接口的设计首先要使软件系统的职责得到合理划分。良好的接口设计可以降低系统各部分的相互依赖，提高组成单元的内聚性，降低组成单元间的耦合程度，从而提高系统的维护性和扩展性。

#### json定义：

JSON(JavaScript Object Notation) 是一种轻量级的数据交换格式。 易于人阅读和编写。同时也易于机器解析和生成。 它基于JavaScript Programming Language, Standard ECMA-262 3rd Edition - December 1999的一个子集。 JSON采用完全独立于语言的文本格式，但是也使用了类似于C语言家族的习惯（包括C, C++, C#, Java, JavaScript, Perl, Python等）。 这些特性使JSON成为理想的数据交换语言。

#### 本次的任务：

通过 API 获取真实、即时的数据,完成一个在命令行界面下天气查询程序，实现以下功能：

输入城市名，返回该城市最新的天气数据；

输入指令，获取帮助信息（一般使用 h 或 help）；

输入指令，获取历史查询信息（一般使用 history）；

输入指令，退出程序的交互（一般使用 quit 或 exit）；

考虑到是否免费、可靠以及是否含有python demo，我们选择的天气API为心知天气API。

首先需要注册获取API秘钥，之后我们可以使用requests调取信息了。

在使用requests之前，需要在终端对其进行安装：pip install requests，然后就可以正常调用了，让我们来看一个例子：

In [ ]:
import requests

result = requests.get('https://api.seniverse.com/v3/weather/grid/now.json')

现在，我们有一个名为result的Response对象。我们可以从这个对象中获取所有我们想要的信息。

get是http协议中最常用的方法之一，另一个是post，我们来看一下他们的区别：

1.GET - 从指定的资源请求数据。

2.POST - 向指定的资源提交要被处理的数据。

接下来，我们使用requests模块发送请求(以北京为例)：

In [17]:
import requests

API = 'https://api.seniverse.com/v3/weather/now.json'

result = requests.get(API, params={
        'key': 'mog6dosgivjijizh',
        'location': '北京',
        'language': 'zh-Hans',
        'unit': 'c'
    }, timeout=1)
print(result.text)

{"results":[{"location":{"id":"WX4FBXXFKE4F","name":"北京","country":"CN","path":"北京,北京,中国","timezone":"Asia/Shanghai","timezone_offset":"+08:00"},"now":{"text":"晴","code":"1","temperature":"21"},"last_update":"2017-08-26T01:45:00+08:00"}]}


key:你注册之后的API秘钥；

location：需要查询的城市名，可以是中文名称，也可以是拼音亦或是城市ID等；

language：语言可以不写，一般默认为简体中文（'zh-Hans'）；

unit：温度单位，可以不写，也可以填‘c’或者‘f’；

timeout:延迟时间。

在向心知天气API发送请求之后，我们观察到，提取的是json格式信息内容，前面我们已经知道，JSON 是一种存储和交换文本信息的语法。它支持多种编程语言，也是目前最流行的数据交换方式。

我们可以引入json库，利用json.loads(requests.text)来将它转换成字典，关于json的作用，我们用一个例子来说明：

In [18]:
import json

json.loads('["foo", {"bar":["baz", null, 1.0, 2]}]')

['foo', {'bar': ['baz', None, 1.0, 2]}]

In [19]:
json.dumps(['foo', {'bar': ('baz', None, 1.0, 2)}])

'["foo", {"bar": ["baz", null, 1.0, 2]}]'

由上可知，json中loads方法是是将从API取得的json数据解码为python可识别的数据，而dumps方法则是将相反。

In [20]:
result_info = json.loads(result.text)
print(result_info)

{'results': [{'last_update': '2017-08-26T01:45:00+08:00',
   'location': {'country': 'CN',
    'id': 'WX4FBXXFKE4F',
    'name': '北京',
    'path': '北京,北京,中国',
    'timezone': 'Asia/Shanghai',
    'timezone_offset': '+08:00'},
   'now': {'code': '1', 'temperature': '21', 'text': '晴'}}]}

然后我们可以从这个字典中取出我们需要的天气信息：

In [11]:
import json
import requests
API = 'https://api.seniverse.com/v3/weather/now.json'

result = requests.get(API, params={
        'key': 'mog6dosgivjijizh',
        'location': '北京',
        'language': 'zh-Hans',
        'unit': 'c'
    }, timeout=1)

result_info = json.loads(result.text)
weather_dic = result_info['results'][0]
city_name = weather_dic['location']['name']
text = weather_dic['now']['text']
temp = weather_dic['now']['temperature']
last_update = weather_dic['last_update']
weather_str = '{}的天气是{},温度是{},最后更新时间：{}'.format(city_name,text,temp,last_update)
print(weather_str)

北京的天气是多云,温度是27,最后更新时间：2017-08-26T14:25:00+08:00


需要说明一点，心知天气的免费用户是不提供县级城市的天气信息的，所以当你输入后，print(result.text)会显示:{"status":"You don't have access to data of this city.","status_code":"AP010006"}权限，输出的结果也会出现KeyError: 'results'的键错误提示。而且，当你输入一个不存在的城市时，也会出现相同的错误，为了避免这种情况发生，我们引入try...except KeyError 语句：

In [ ]:
......
try:
    result_info = json.loads(result.text)
    weather_dic = result_info['results'][0]
    city_name = weather_dic['location']['name']
    text = weather_dic['now']['text']
    temp = weather_dic['now']['temperature']
    last_update = weather_dic['last_update']
    weather_str = '{}的天气是{},温度是{},最后更新时间：{}'.format(city_name,text,temp,last_update)
    rint(weather_str)
except KeyError:
    pass

except KeyError后面接我们希望当此类错误出现后应该执行的程序段。

借用requests 向API发出请求并处理接收回来的数据信息基本已经完成，接下来我们再来完成本章的基础任务：

In [ ]:
while True:
    user_input = input('请输入需要查询的城市：')
    if user_input in ['h','help']:
            print('''
                输入城市名,查询该该城市的天气;
                输入help,获取帮助文档;
                输入history,获取查询历史;
                输入quit,退出天气查询系统.
            ''')
        elif user_input in ['q','quit']:
            print('已退出天气查询系统！')
            break
        elif user_input == 'history':
            print('你查询的历史信息为：')
            for item in history_list:
                print(item)
        else:
            print('你输入的信息有误，请重新输入！')

最后，为了体现程序的易读性和健壮性，我们把程序包装成函数：

In [ ]:
import requests
import json

url = 'https://api.seniverse.com/v3/weather/daily.json'
#def fetchWeather(location):

def getweather(location):
    result = requests.get(url, params={
            'key': 'mog6dosgivjijizh',
            'location': location,
            'language': 'zh-Hans',
            'unit': ''
        }, timeout=2)
    return json.loads(result.text) 

def print_weather(result_info):
    city_name = result_info['results'][0]['location']['name']
    date = result_info['results'][0]['daily'][0]['date']
    text_day = result_info['results'][0]['daily'][0]['text_day']
    text_night =result_info['results'][0]['daily'][0]['text_night']
    high = result_info['results'][0]['daily'][0]['high']
    low = result_info['results'][0]['daily'][0]['low']
    wind_direction =result_info['results'][0]['daily'][0]['wind_direction']
    wind_scale = result_info['results'][0]['daily'][0]['wind_scale']
    weather_info = '{}：{}的天气：白天{}，夜晚{}，温度：{}~{}，{}风{}级' .format(date,city_name,text_day,text_night,high,low,wind_direction,wind_scale)
    return weather_info
history_list = []

while True:
    user_input = input('输入一个城市：')
    try:
        result_info = getweather(user_input)
        print(print_weather(result_info))
        history_list.append(print_weather(result_info))
    except KeyError:
        if user_input in ['h','help']:
            print('''
                输入城市名,查询该该城市的天气;
                输入help,获取帮助文档;
                输入history,获取查询历史;
                输入quit,退出天气查询系统.
			    ''')
        elif user_input == 'history':
            if not history_list:
                print('你还没有查询历史。')
            else:
                for item in history_list:
                    print(item)
        elif user_input in ['q','quit']:
            print('退出天气查询系统')
            break
        else：
        print('你输入的信息有误，请重新输入。')